<h1 align="center">📘 Research Agent Analyst</h1>

<p align="center">
A fully automated AI agent that reads research PDFs, extracts and restructures
their content, builds context, summarizes them in a student-friendly way, and
evaluates the output for clarity and correctness — all using Vertex AI and
Google’s Agent Development Kit (ADK).
</p>


# 🔧 Environment Setup
Before building our research agent, we must install all required Google ADK and
Vertex AI packages. These libraries allow our notebook to deploy agents, interact
with the Reasoning Engine, and process PDFs.  
This section ensures your environment has all the dependencies needed for the
rest of the pipeline.


In [ ]:
!pip install google-cloud-aiplatform[adk,agent_engines] google-adk


INFO: pip is looking at multiple versions of opentelemetry-instrumentation to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.1
    Uninstalling wrapt-2.0.1:
      Successfully uninstalled wrapt-2.0.1


# 📦 Importing Required Libraries
Here we load all the Python libraries used throughout the project.This includes Vertex AI, ADK components, PDF readers, and utility modules.  


In [ ]:
import os
import random
import time
import vertexai
from vertexai import agent_engines

print("✅ Imports completed successfully")

✅ Imports completed successfully


# 🔐 Configuring API Keys and Project Settings
In this section, we connect our notebook to the correct Google Cloud project.  
We set environment variables for:
- Project ID  
- API keys  
- Region  
These values tell Vertex AI and ADK where to deploy and run our agent.


In [ ]:
from google.colab import userdata
import os
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [ ]:
## Set your PROJECT_ID
PROJECT_ID = "my-research-agent-478404"  # TODO: Replace with your project ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

if PROJECT_ID == "your-project-id" or not PROJECT_ID:
    raise ValueError("⚠️ Please replace 'your-project-id' with your actual Google Cloud Project ID.")

print(f"✅ Project ID set to: {PROJECT_ID}")

✅ Project ID set to: my-research-agent-478404


In [ ]:
!gcloud auth activate-service-account --key-file="/content/my-research-agent-478404-594c8dc978b0.json"
!gcloud config set project my-research-agent-478404


Activated service account credentials for: [research-agent-sa@my-research-agent-478404.iam.gserviceaccount.com]
Updated property [core/project].


In [ ]:
import vertexai
import os

PROJECT_ID = "my-research-agent-478404"
REGION = "us-east4"
BUCKET = "gs://my-research-agent-staging"

# MUST BE SET BEFORE vertexai.init()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/my-research-agent-478404-594c8dc978b0.json"

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET,
)

print("Vertex AI initialized with service account JSON")


Vertex AI initialized with service account JSON


In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/my-research-agent-478404-594c8dc978b0.json"

print("Credentials set.")


Credentials set.


# 📁 Preparing Project Structure
We create a dedicated folder (`research_agent/`) to store all files for the
research agent — such as Python scripts, requirements, environment configs, and
agent engine files.  



In [ ]:
!mkdir -p research_agent


In [ ]:
%%writefile research_agent/requirements.txt
google-adk
google-cloud-aiplatform[adk,agent_engines]>=1.111
opentelemetry-instrumentation-google-genai
pypdf
requests
google-genai



Writing research_agent/requirements.txt


In [ ]:
%%writefile research_agent/.env

# https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations#global-endpoint
GOOGLE_CLOUD_PROJECT="my-research-agent-478404"
GOOGLE_CLOUD_LOCATION="us-east4"
GOOGLE_GENAI_USE_VERTEXAI=1

Writing research_agent/.env


In [ ]:
%%writefile research_agent/.agent_engine_config.json
{
  "min_instances": 0,
  "max_instances": 1,
  "resource_limits": {
    "cpu": "1",
    "memory": "2Gi"
  }
}


Writing research_agent/.agent_engine_config.json


# 🤖 Building the Research Agent
In this section, we define the entire agent pipeline:
- PDF reader tool  
- Document reader agent  
- Context builder agent  
- Summarizer agent  
- Evaluator agent  
- Root orchestrator agent  
This is the heart of the project — it automates the full research-paper workflow.


In [41]:
%%writefile research_agent/agent.py
import os
import tempfile
import logging

from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import AgentTool, load_memory, preload_memory

from pypdf import PdfReader
from google.cloud import storage


# ============================================================
# Logging / Observability
# ============================================================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
)
logger = logging.getLogger("research_capstone_agent")

# ============================================================
# App Constants
# ============================================================
APP_NAME = "ResearchAgentCapstone"
USER_ID = "capstone_user"

# ============================================================
# Retry config for LLM calls
# ============================================================
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=5,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# ============================================================
# Session & Memory Services
# ============================================================
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

# ============================================================
# Custom Tool: PDF Reader (local path or GCS)
# ============================================================
def pdf_reader_tool(file_path: str) -> dict:
    """
    Reads a PDF either from local path or GCS (gs://bucket/path.pdf)
    and returns the extracted text.

    Args:
        file_path: Local path or GCS URI to the PDF.

    Returns:
        dict: {"status": "success", "text": ""} or
              {"status": "error", "error": ""} on failure.
    """
    try:
        # Create a temporary local file to store the PDF
        temp_pdf = tempfile.NamedTemporaryFile(
            delete=False, suffix=".pdf"
        ).name

        # If path is GCS, download first
        if file_path.startswith("gs://"):
            storage_client = storage.Client()
            path_no_scheme = file_path.replace("gs://", "", 1)
            bucket_name = path_no_scheme.split("/")[0]
            blob_name = "/".join(path_no_scheme.split("/")[1:])

            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)
            blob.download_to_filename(temp_pdf)
            local_path = temp_pdf
        else:
            # Treat as local file path
            local_path = file_path

        # Extract text from PDF
        reader = PdfReader(local_path)
        text = "\n".join(page.extract_text() or "" for page in reader.pages)

        return {"status": "success", "text": text}

    except Exception as e:
        return {"status": "error", "error": str(e)}

# ============================================================
# Sub-Agents (LlmAgents) — Multi-Agent Pipeline
# ============================================================

# 1) Document Reader – raw text -> structured markdown
document_reader = LlmAgent(
    name="document_reader",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    instruction="""
You receive the FULL TEXT of a research paper.

Transform it into CLEAN, STRUCTURED MARKDOWN:

- Title
- Authors (if available)
- Abstract
- Sections with headings
- Important equations (LaTeX if possible)
- Tables in text form

RULES:
- Do NOT summarize.
- Do NOT invent any content.
- Only clean, structure, and format what is present in the text.
""",
    input_args_schema={
        "text": {
            "type": "string",
            "description": "Raw full text extracted from the PDF.",
        }
    },
)

# 2) Context Builder – markdown -> 400–600 word compressed context
context_builder = LlmAgent(
    name="context_builder",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    instruction="""
You receive structured markdown for a research paper.

Compress it into 400–600 words while preserving:
- Motivation / problem
- Methods / approach
- Datasets / inputs
- Results
- Conclusions

RULES:
- Do NOT invent details.
- Keep wording clear, dense, and technically accurate.
""",
    input_args_schema={
        "markdown": {
            "type": "string",
            "description": "Structured markdown produced by document_reader.",
        }
    },
)

# 3) Summarizer – context -> student-friendly summary
summarizer = LlmAgent(
    name="summarizer",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    instruction="""
You receive a 400–600 word context about a research paper.

Produce a STUDENT-FRIENDLY summary with these sections:

1. Title
2. Problem
3. Background / Motivation
4. Method / Model
5. Dataset / Inputs
6. Results (include important numbers if present)
7. Strengths
8. Limitations
9. Key Takeaways (5–10 bullet points)
10. Overall Conclusion

RULES:
- Length: 300–600 words.
- Use clear, simple English suitable for a CS/AI student.
- Do NOT hallucinate information that is not supported by the context.
""",
    input_args_schema={
        "context": {
            "type": "string",
            "description": "Compressed technical context produced by context_builder.",
        }
    },
)

# 4) Evaluator – context + summary -> feedback bullets
evaluator = LlmAgent(
    name="evaluator",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    instruction="""
You receive:
- A compressed context of a research paper.
- A candidate student-focused summary.

Evaluate ONLY on:
- Technical correctness
- Clarity
- Completeness (are key ideas covered?)

Return:
- SHORT bullet-point feedback listing any issues or confirming high quality.

Do NOT rewrite the summary or restate the full paper.
""",
    input_args_schema={
        "context": {
            "type": "string",
            "description": "Compressed context for the paper.",
        },
        "summary": {
            "type": "string",
            "description": "Draft student-friendly summary to evaluate.",
        },
    },
)

# ============================================================
# Wrap Sub-Agents as Tools (Agent-to-Agent communication)
# ============================================================
document_reader_tool = AgentTool(agent=document_reader)
context_builder_tool = AgentTool(agent=context_builder)
summarizer_tool = AgentTool(agent=summarizer)
evaluator_tool = AgentTool(agent=evaluator)

# ============================================================
# Automatic Memory Callback
# ============================================================
async def auto_save_to_memory(callback_context):
    """
    Automatically save the current session into memory after each agent turn.

    Uses callback_context._invocation_context to access:
    - memory_service
    - current session
    """
    try:
        invocation_ctx = callback_context._invocation_context
        session = invocation_ctx.session
        mem_service = invocation_ctx.memory_service

        if session is not None and mem_service is not None:
            await mem_service.add_session_to_memory(session)
            logger.info("auto_save_to_memory: session %s saved to memory.", session.id)
        else:
            logger.warning("auto_save_to_memory: session or memory_service is None.")
    except Exception as e:
        logger.exception("auto_save_to_memory failed: %s", e)

# ============================================================
# Root Research Agent (Orchestrator + Memory)
# ============================================================
research_agent = LlmAgent(
    name="research_analyst",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    instruction="""
You are the RESEARCH ANALYST AGENT.

You must follow this pipeline when the user wants a research paper analyzed:

1) Use `pdf_reader_tool(file_path=...)` to extract raw text from the PDF.
   - If the tool returns status != "success", briefly explain the error and stop.

2) Call `document_reader(text = <raw_pdf_text>)`
   to produce CLEAN MARKDOWN.

3) Call `context_builder(markdown = <markdown>)`
   to get a 400–600 word compressed technical context.

4) Call `summarizer(context = <context>)`
   to produce a student-friendly summary.

5) Call `evaluator(context = <context>, summary = <draft_summary>)`
   and use the evaluator's feedback internally to fix obvious problems.

FINAL RESPONSE RULES:
- Return ONLY the final improved summary.
- Do NOT include raw PDF text.
- Do NOT include the intermediate markdown.
- Do NOT include evaluator feedback.
- Length: 300–600 words.
- Make it clear, structured, and easy to read for a CS/AI student.

MEMORY USAGE:
- Use `load_memory` when you need to recall past user preferences
  (for example: preferred level of detail, focus areas like math vs. experiments).
- `preload_memory` can be used to automatically load relevant memories
  before responding in a new session.

You are part of a multi-agent system and must rely on the tools and
sub-agents instead of doing everything yourself.
""",
    tools=[
        pdf_reader_tool,         # Custom Python tool
        document_reader_tool,    # LLM sub-agent
        context_builder_tool,    # LLM sub-agent
        summarizer_tool,         # LLM sub-agent
        evaluator_tool,          # LLM sub-agent
        load_memory,             # Memory tool (reactive)
        preload_memory,          # Memory tool (proactive)
    ],
    after_agent_callback=auto_save_to_memory,  # automatic memory saving
)

# ============================================================
# Runner (Agent + Sessions + Memory)
# ============================================================
runner = Runner(
    agent=research_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)


logger.info("Research capstone agent initialized with sessions, memory, and auto-save.")


Overwriting research_agent/agent.py


# 🚀 Deploying Agent Engine to Vertex AI
We package all files, build the agent engine, and deploy it to Google Cloud.  
Once deployed, the agent becomes available as a cloud service you can query
remotely.


In [ ]:
!adk deploy agent_engine \
 --project=$PROJECT_ID \
 --region=us-east4 \
 research_agent \
 --agent_engine_config_file=research_agent/.agent_engine_config.json


# 🔗 Connecting to Deployed Agent
After deployment, we fetch the agent from Vertex AI.  
This allows us to send queries, upload PDFs, and run the agent pipeline in the cloud.


In [27]:
deployed_region = "us-east4"
# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=deployed_region)

# Get the most recently deployed agent
agents_list = list(agent_engines.list())
if agents_list:
    remote_agent = agents_list[0]  # Get the first (most recent) agent
    client = agent_engines
    print(f"✅ Connected to deployed agent: {remote_agent.resource_name}")
else:
    print("❌ No agents found. Please deploy first.")

✅ Connected to deployed agent: projects/610233153602/locations/us-east4/reasoningEngines/6225909825490911232


# 📤 Uploading PDF to Google Cloud Storage
We upload a PDF document to our GCS bucket.  
The agent will read this file directly during processing.


In [29]:
!gsutil cp "/content/ase_10.pdf" gs://my-research-agent-staging/uploads/ase10.pdf


CommandException: No URLs matched: /content/ase_10.pdf


#Sample Test

In [31]:
async for event in remote_agent.async_stream_query(
    user_id="u1",
    message="Hello, what can you do?"
):
    print(event)


{'model_version': 'gemini-2.0-flash', 'content': {'parts': [{'text': 'I can read research PDFs, extract text, create clean markdown, build context, produce summaries, and evaluate the summaries. How can I help you today?\n'}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 32, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 32}], 'prompt_token_count': 398, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 398}], 'total_token_count': 430, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.24299049377441406, 'invocation_id': 'e-83c6f23f-8702-4610-b48a-4edd86ac8c52', 'author': 'research_analyst', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'id': '47086bd3-8415-4be9-82d2-be14306f7598', 'timestamp': 1763770207.745959}


# 🧠 Running the Research Agent End-to-End
Finally, we send a query to the deployed agent along with the PDF.  
The agent:
1. Reads the PDF  
2. Cleans the text  
3. Builds a context  
4. Summarizes  
5. Evaluates & improves  
6. Returns the final summary  

This is where everything comes together and you see the full pipeline in action.


In [ ]:
async for event in remote_agent.async_stream_query(
    user_id="u1",
    message=(
        "Process this PDF and give me the final summary. "
        "file_path=gs://my-research-agent-staging/uploads/ase10.pdf"
    ),
):
    print(event)


# Deleting Resources Created

In [ ]:
from vertexai import agent_engines
import vertexai

vertexai.init(project="my-research-agent-478404", location="us-east4") #east4 also delete

agents = list(agent_engines.list())
for a in agents:
    print("Deleting:", a.resource_name)
    agent_engines.delete(a.resource_name, force=True)
